In [1]:
from typing import *

In [47]:
def print_graph (g) :
    for v in g :
        edges = g[v] 
        print(f"v: {v} edges: {', '.join([(f'({to} w: {w})') for to, w in edges])}")

In [48]:
def make_graph_from_tuples (tuples) :
    g = {}
    for v_from, v_to, weight in tuples :
        if v_from in g :
            g[v_from].append((v_to, weight))
        else :
            g[v_from] = [(v_to, weight)]
    return g

In [103]:
g = make_graph_from_tuples([
    (0, 1, 1),
    (1, 2, 3),
    (3, 2, 0.5),
    (1, 3, 1),
    (1, 2, 10),
])

In [104]:
print_graph(g)

v: 0 edges: (1 w: 1)
v: 1 edges: (2 w: 3), (3 w: 1), (2 w: 10)
v: 3 edges: (2 w: 0.5)


In [105]:
def get_vertices (g) :
    result = []
    result.extend(g.keys())
    for v in g :
        result.extend([v_to for v_to, w in g[v]])
    return set(result)

In [106]:
def invert_edges (g) :
    inverted_g = {}
    for v_to in g :
        for v_from, w in g[v_to] :
            if v_from in inverted_g :
                inverted_g[v_from].append((v_to, w))
            else :
                inverted_g[v_from] = [(v_to, w)]
    return inverted_g

In [107]:
print_graph(invert_edges(g))

v: 1 edges: (0 w: 1)
v: 2 edges: (1 w: 3), (1 w: 10), (3 w: 0.5)
v: 3 edges: (1 w: 1)


In [150]:
import math

def BellmanFord(g, s):
    t = [[math.inf if v != s else 0.0 for v in list(get_vertices(g))]]
    m = [[None] * len(get_vertices(g))]

    g_with_inverted_edges = invert_edges (g)

    for k in range(1, len(get_vertices(g)) + 1):
        
        t.append(list(t[-1]))
        m.append(list(m[-1]))

        for i, v in enumerate(get_vertices(g)) :
            if v == s :
                t[k][i] = 0
                continue

            if v not in g_with_inverted_edges : # no edges from this one
                print(f"skip {v}")
                continue

            for v_from, w in g_with_inverted_edges[v] :
                print(f"k = {k}: v = {v} (t={t[k][i]}) v_from = {v_from} (t={t[k - 1][list(get_vertices(g)).index(v_from)]}) w = {w}")
                if t[k - 1][list(get_vertices(g)).index(v_from)] + w < t[k][i] :
                    print(f"update {v} (new: {v_from} --> {v})")
                    t[k][i] = t[k][list(get_vertices(g)).index(v_from)] + w
                    m[k][i] = v_from

    # check for conservativity

    for i in range(len(get_vertices(g))) :
        if t[-1][i] != t[-2][i] :
            raise ValueError(f"Conservativity not preserved! (weight of {list(g.keys())[i]} changed)")

    return t, m

In [151]:
print_graph(g)
print("----")
print_graph(invert_edges(g))

v: 0 edges: (1 w: 1)
v: 1 edges: (2 w: 3), (3 w: 1), (2 w: 10)
v: 3 edges: (2 w: 0.5)
----
v: 1 edges: (0 w: 1)
v: 2 edges: (1 w: 3), (1 w: 10), (3 w: 0.5)
v: 3 edges: (1 w: 1)


In [152]:
BellmanFord(g, 0)

k = 1: v = 1 (t=inf) v_from = 0 (t=0.0) w = 1
update 1 (new: 0 --> 1)
k = 1: v = 2 (t=inf) v_from = 1 (t=inf) w = 3
k = 1: v = 2 (t=inf) v_from = 1 (t=inf) w = 10
k = 1: v = 2 (t=inf) v_from = 3 (t=inf) w = 0.5
k = 1: v = 3 (t=inf) v_from = 1 (t=inf) w = 1
k = 2: v = 1 (t=1) v_from = 0 (t=0) w = 1
k = 2: v = 2 (t=inf) v_from = 1 (t=1) w = 3
update 2 (new: 1 --> 2)
k = 2: v = 2 (t=4) v_from = 1 (t=1) w = 10
k = 2: v = 2 (t=4) v_from = 3 (t=inf) w = 0.5
k = 2: v = 3 (t=inf) v_from = 1 (t=1) w = 1
update 3 (new: 1 --> 3)
k = 3: v = 1 (t=1) v_from = 0 (t=0) w = 1
k = 3: v = 2 (t=4) v_from = 1 (t=1) w = 3
k = 3: v = 2 (t=4) v_from = 1 (t=1) w = 10
k = 3: v = 2 (t=4) v_from = 3 (t=2) w = 0.5
update 2 (new: 3 --> 2)
k = 3: v = 3 (t=2) v_from = 1 (t=1) w = 1
k = 4: v = 1 (t=1) v_from = 0 (t=0) w = 1
k = 4: v = 2 (t=2.5) v_from = 1 (t=1) w = 3
k = 4: v = 2 (t=2.5) v_from = 1 (t=1) w = 10
k = 4: v = 2 (t=2.5) v_from = 3 (t=2) w = 0.5
k = 4: v = 3 (t=2) v_from = 1 (t=1) w = 1


([[0.0, inf, inf, inf],
  [0, 1, inf, inf],
  [0, 1, 4, 2],
  [0, 1, 2.5, 2],
  [0, 1, 2.5, 2]],
 [[None, None, None, None],
  [None, 0, None, None],
  [None, 0, 1, 1],
  [None, 0, 3, 1],
  [None, 0, 3, 1]])